This is the minimum amount of code I need to run to fit a model. If I was building a framework I might make this simpler, but for research purposes it's not worth simplifying any further. The train_model function probably needs the most cleanup and even then most of the icky stuff is my progression tracking.

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
# ^---not strictly necessary, but what data scientist doesn't automatically import np and plt
import torch
import torch.nn as nn
from kural_core.models import *
from kural_core.data_processing import *
from kural_core.train_model import train_model

PATH = r'D:\Code\kural_work\\'

In [ ]:
ds = BoundingLandmarksDataset(crop_img_tensor,xy_tensor) ## see kural_core/data_processing.py for constructing datasets
train_idx, val_idx = RandomIndicesForKFoldValidation(len(ds), 0, K=5, rand_seed=42, nORp=1) ## see kural_core/data_processing.py
# I have never actually used the above function for k-fold validation, merely for creating a single validation set.
bs = 9 # batch size. You can adjust this based on GPU memory
train_dl = torch.utils.data.DataLoader(ds,batch_size=bs, sampler=SubsetSampler(train_idx, shuffle=True))
val_dl = torch.utils.data.DataLoader(ds,batch_size=5*bs, sampler=SubsetSampler(val_idx))
model = BoundingPointsModel().cuda() ## see kural_core/models.py
opt = optim.Adam(model.parameters(),)
train_loss = []
val_loss = []
accuracy = [] # only functions for classification problems
criterion = nn.MSELoss()

In [ ]:
# see kural_core/train_model.py
model, _tl, _vl, _acc = train_model(model, train_dl, val_dl, opt, criterion, lr_start=1e-4, lr_end=1, epochs=1000, cycles=1, is_classification=True, lr_cycle='sin')
train_loss += _tl
val_loss += _vl
accuracy += _acc # remember to take this out for non-classification problems
torch.save(model.state_dict(), r'D:\Code\kural_work\actin_classifier_model.pth')